In [2]:
from gensim import corpora, models, similarities, matutils
import itertools
import numpy as np
import pandas as pd

In [3]:
dropbox = "/Users/mr/Dropbox/moviemeta/"

In [9]:
wiki_lda = models.LdaModel.load(dropbox +'lda_wiki.model')
wiki_corpus = corpora.MmCorpus(dropbox +'lda_wiki.corpus')
wiki_dict = corpora.Dictionary.load(dropbox +'lda_wiki.dict')
wiki_meta_df = pd.read_csv(dropbox + 'wiki_meta_df.csv')

In [10]:
wiki_topic_matrix = matutils.corpus2dense(lda[corpus], num_terms=30)
wiki_topic_df = pd.DataFrame(np.ndarray.transpose(wiki_topic_matrix))
#topic_df['movie_title'] = pd.Series(movies, index=topic_df.index)

Now we have created a dataframe with the topic distribution for every movie. We have 20 topics, columns 0 - 19. The values represent how prominently a topic features in a movie.

In [14]:
wiki_topic_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0,0,0,0,0.000000,0.00000,0.000000,0.000000,0,0,...,0,0,0.000000,0,0,0,0.641651,0.000000,0.000000,0
1,0,0,0,0,0.053756,0.00000,0.000000,0.000000,0,0,...,0,0,0.181564,0,0,0,0.000000,0.000000,0.754680,0
2,0,0,0,0,0.000000,0.32694,0.000000,0.163939,0,0,...,0,0,0.000000,0,0,0,0.000000,0.000000,0.327525,0
3,0,0,0,0,0.000000,0.00000,0.301251,0.000000,0,0,...,0,0,0.000000,0,0,0,0.000000,0.000000,0.000000,0
4,0,0,0,0,0.000000,0.00000,0.000000,0.929066,0,0,...,0,0,0.000000,0,0,0,0.000000,0.067812,0.000000,0


In [18]:
wiki_meta_df.head()

,Unnamed: 0,title,year,countries
0,0,Well-Founded Fear,2000,United States
1,2,Thenali,2000,India
2,6,The Season of Men,2000,"France, Tunisia"
3,7,Beautiful Mistake,2000,Wales
4,8,Ready to Rumble,2000,United States


In [31]:
wiki_topic_df = pd.concat([wiki_topic_df, wiki_meta_df], axis=1) 
wiki_topic_df.head()

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,Unnamed: 0,title,year,countries
0,0,0,0,0,0.000000,0.00000,0.000000,0.000000,0,0,...,0,0,0.641651,0.000000,0.000000,0,0,Well-Founded Fear,2000,United States
1,0,0,0,0,0.053756,0.00000,0.000000,0.000000,0,0,...,0,0,0.000000,0.000000,0.754680,0,2,Thenali,2000,India
2,0,0,0,0,0.000000,0.32694,0.000000,0.163939,0,0,...,0,0,0.000000,0.000000,0.327525,0,6,The Season of Men,2000,"France, Tunisia"
3,0,0,0,0,0.000000,0.00000,0.301251,0.000000,0,0,...,0,0,0.000000,0.000000,0.000000,0,7,Beautiful Mistake,2000,Wales
4,0,0,0,0,0.000000,0.00000,0.000000,0.929066,0,0,...,0,0,0.000000,0.067812,0.000000,0,8,Ready to Rumble,2000,United States


In [22]:
topmovies_df=pd.read_csv(dropbox + "Top500_since2001.csv")

In [28]:
us_df = topmovies_df[topmovies_df.country=='United States']
us_df.head()

,country,year,movie
4139,United States,2014,American Sniper
4140,United States,2014,The Hunger Games: Mockingjay - Part 1
4141,United States,2014,Guardians of the Galaxy
4142,United States,2014,Captain America: The Winter Soldier
4143,United States,2014,The LEGO Movie


In [33]:
us_wiki_df = wiki_topic_df[wiki_topic_df['title'].isin(us_df.movie.tolist())]

In [35]:
print us_wiki_df.shape
us_wiki_df.head()

(4413, 34)


,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,Unnamed: 0,title,year,countries
8,0,0.361445,0.00000,0,0,0,0,0.000000,0.000000,0,...,0.466526,0.000000,0,0.000000,0,0.000000,13,Aberdeen,2000,"Norway, Sweden, United Kingdom"
14,0,0.023080,0.00000,0,0,0,0,0.286480,0.223847,0,...,0.000000,0.114903,0,0.064416,0,0.256945,21,Tears of the Black Tiger,2000,Thailand
20,0,0.000000,0.00000,0,0,0,0,0.000000,0.000000,0,...,0.000000,0.000000,0,0.000000,0,0.000000,27,Help! I'm a Fish,2000,"Denmark, Germany, Ireland"
25,0,0.000000,0.01731,0,0,0,0,0.037113,0.000000,0,...,0.000000,0.000000,0,0.087541,0,0.000000,32,Bang Rajan,2000,Thailand
32,0,0.000000,0.00000,0,0,0,0,0.000000,0.000000,0,...,0.128445,0.000000,0,0.000000,0,0.000000,44,Better Than Sex,2000,Australia / France
